In [1]:
! git clone https://github.com/nachi-hebbar/Transfer-Learning-ResNet-Keras.git

Cloning into 'Transfer-Learning-ResNet-Keras'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (6/6), 476.00 KiB | 2.97 MiB/s, done.


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

2023-02-17 09:42:04.029579: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-17 09:42:04.246432: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-17 09:42:04.246473: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-17 09:42:05.191509: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [3]:
my_dir = r'/home/ubuntu/Downloads/cv/number_plate data/'

train_dir = os.path.join(my_dir, 'train')

val_dir =  os.path.join(my_dir, 'val')

In [4]:
img_height,img_width=224,224
batch_size=32
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  train_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 359627 files belonging to 36 classes.
Using 287702 files for training.


2023-02-17 09:42:21.225174: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-17 09:42:21.225733: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-17 09:42:21.225763: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ubuntu-OptiPlex-3040): /proc/driver/nvidia/version does not exist
2023-02-17 09:42:21.229876: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  val_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 87988 files belonging to 36 classes.
Using 17597 files for validation.


In [6]:
class_names = train_ds.class_names
print(class_names)

['class_0', 'class_1', 'class_2', 'class_3', 'class_4', 'class_5', 'class_6', 'class_7', 'class_8', 'class_9', 'class_A', 'class_B', 'class_C', 'class_D', 'class_E', 'class_F', 'class_G', 'class_H', 'class_I', 'class_J', 'class_K', 'class_L', 'class_M', 'class_N', 'class_O', 'class_P', 'class_Q', 'class_R', 'class_S', 'class_T', 'class_U', 'class_V', 'class_W', 'class_X', 'class_Y', 'class_Z']


In [7]:
resnet_model = Sequential()

pretrained_model= tf.keras.applications.ResNet50(include_top=False,
                   input_shape=(224,224,3),
                   pooling='avg',classes=36,
                   weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False

resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(36, activation='softmax'))

In [8]:
resnet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 module_wrapper (ModuleWrapp  (None, 2048)             0         
 er)                                                             
                                                                 
 module_wrapper_1 (ModuleWra  (None, 512)              1049088   
 pper)                                                           
                                                                 
 module_wrapper_2 (ModuleWra  (None, 36)               18468     
 pper)                                                           
                                                                 
Total params: 24,655,268
Trainable params: 1,067,556
Non-trainable params: 23,587,712
____________________________________

In [9]:
resnet_model.compile(optimizer=Adam(lr=0.001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

/home/ubuntu/.local/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
epochs = 1
history = resnet_model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/10


2023-02-17 09:42:31.157518: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 19267584 exceeds 10% of free system memory.
2023-02-17 09:42:31.175495: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 20313600 exceeds 10% of free system memory.
2023-02-17 09:42:31.191518: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 102760448 exceeds 10% of free system memory.
2023-02-17 09:42:31.201782: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 19267584 exceeds 10% of free system memory.
2023-02-17 09:42:31.411547: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 102760448 exceeds 10% of free system memory.


5847/8991 [==================>...........] - ETA: 3:51:33 - loss: 0.2155 - accuracy: 0.9337

In [ ]:
fig1 = plt.gcf()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.axis(ymin=0.4,ymax=1)
plt.grid()
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.grid()
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()

In [ ]:
import cv2
image=cv2.imread('/content/drive/MyDrive/Number_Plate_Classification/data/val/class_6/class_6_1.jpg') # Class-6
image_resized= cv2.resize(image, (img_height,img_width))
image=np.expand_dims(image_resized,axis=0)
print(image.shape)


In [ ]:
pred=resnet_model.predict(image)
print(pred)

In [ ]:
output_class=class_names[np.argmax(pred)]
print("The predicted class is", output_class) # class-6

In [ ]:
resnet_model.save('Resnet_18_Num_Plate')

In [ ]:

import pickle
pickle.dumps(resnet_model)

In [ ]:
a=os.listdir('/content/drive/MyDrive/Number_Plate_Classification/data/train')

In [ ]:
print(a)

In [ ]:
import albumentations as A
import cv2
import os
import numpy as np




def Saving_files(root_path):
    for folder in os.listdir(root_path):
        sub_path=root_path+'/'+folder
        # print(sub_path)
        # os.makedirs(desti_path+'/'+folder)
            
        img=cv2.imread(sub_path)
        image_resized= cv2.resize(img, (img_height,img_width))
        image=np.expand_dims(image_resized,axis=0)
        pred=resnet_model.predict(image)
        output_class=class_names[np.argmax(pred)]
        print("The predicted class is", output_class)
        if output_class=='class_0':
          cv2.imwrite('/content/drive/MyDrive/Number_Plate_Classification/data/train/class_0'+'/'+folder,img)
          print('stored at class_0 ')
        
        elif output_class=='class_1':
          cv2.imwrite('/content/drive/MyDrive/Number_Plate_Classification/data/train/class_1'+'/'+folder,img)
          print('stored at class_2 ')

        elif output_class=='class_2':
          cv2.imwrite('/content/drive/MyDrive/Number_Plate_Classification/data/train/class_2'+'/'+folder,img)
          print('stored at class_2 ')

        elif output_class=='class_3':
          cv2.imwrite('/content/drive/MyDrive/Number_Plate_Classification/data/train/class_3'+'/'+folder,img)
          print('stored at class_3 ')
        
        elif output_class=='class_4':
          cv2.imwrite('/content/drive/MyDrive/Number_Plate_Classification/data/train/class_4'+'/'+folder,img)
          print('stored at class_4 ')
        elif output_class=='class_5':
          cv2.imwrite('/content/drive/MyDrive/Number_Plate_Classification/data/train/class_5'+'/'+folder,img)
          print('stored at class_5 ')
        elif output_class=='class_6':
          cv2.imwrite('/content/drive/MyDrive/Number_Plate_Classification/data/train/class_6'+'/'+folder,img)
          print('stored at class_6 ')
        elif output_class=='class_7':
          cv2.imwrite('/content/drive/MyDrive/Number_Plate_Classification/data/train/class_7'+'/'+folder,img)
          print('stored at class_7 ')

        elif output_class=='class_8':
          cv2.imwrite('/content/drive/MyDrive/Number_Plate_Classification/data/train/class_8'+'/'+folder,img)
          print('stored at class_8 ')

        elif output_class=='class_9':
          cv2.imwrite('/content/drive/MyDrive/Number_Plate_Classification/data/train/class_9'+'/'+folder,img)
          print('stored at class_9 ')

        elif output_class=='class_A':
          cv2.imwrite('/content/drive/MyDrive/Number_Plate_Classification/data/train/class_A'+'/'+folder,img)
          print('stored at class_A ')
        elif output_class=='class_B':
          cv2.imwrite('/content/drive/MyDrive/Number_Plate_Classification/data/train/class_B'+'/'+folder,img)
          print('stored at class_B ')
        elif output_class=='class_C':
          cv2.imwrite('/content/drive/MyDrive/Number_Plate_Classification/data/train/class_C'+'/'+folder,img)
          print('stored at class_C ')

        elif output_class=='class_D':
          cv2.imwrite('/content/drive/MyDrive/Number_Plate_Classification/data/train/class_D'+'/'+folder,img)
          print('stored at class_D ')
        elif output_class=='class_E':
          cv2.imwrite('/content/drive/MyDrive/Number_Plate_Classification/data/train/class_E'+'/'+folder,img)
          print('stored at class_E ')
        elif output_class=='class_F':
          cv2.imwrite('/content/drive/MyDrive/Number_Plate_Classification/data/train/class_F'+'/'+folder,img)
          print('stored at class_F ')
        elif output_class=='class_G':
          cv2.imwrite('/content/drive/MyDrive/Number_Plate_Classification/data/train/class_G'+'/'+folder,img)
          print('stored at class_G ')
        elif output_class=='class_H':
          cv2.imwrite('/content/drive/MyDrive/Number_Plate_Classification/data/train/class_H'+'/'+folder,img)
          print('stored at class_H ')
        elif output_class=='class_I':
          cv2.imwrite('/content/drive/MyDrive/Number_Plate_Classification/data/train/class_I'+'/'+folder,img)
          print('stored at class_I ')
        elif output_class=='class_J':
          cv2.imwrite('/content/drive/MyDrive/Number_Plate_Classification/data/train/class_J'+'/'+folder,img)
          print('stored at class_J ')
        elif output_class=='class_K':
          cv2.imwrite('/content/drive/MyDrive/Number_Plate_Classification/data/train/class_K'+'/'+folder,img)
          print('stored at class_K ')
        elif output_class=='class_L':
          cv2.imwrite('/content/drive/MyDrive/Number_Plate_Classification/data/train/class_L'+'/'+folder,img)
          print('stored at class_L ')
        elif output_class=='class_M':
          cv2.imwrite('/content/drive/MyDrive/Number_Plate_Classification/data/train/class_M'+'/'+folder,img)
          print('stored at class_M ')
        elif output_class=='class_N':
          cv2.imwrite('/content/drive/MyDrive/Number_Plate_Classification/data/train/class_N'+'/'+folder,img)
          print('stored at class_N ')
        elif output_class=='class_O':
          cv2.imwrite('/content/drive/MyDrive/Number_Plate_Classification/data/train/class_P'+'/'+folder,img)
          print('stored at class_O ')

        elif output_class=='class_Q':
          cv2.imwrite('/content/drive/MyDrive/Number_Plate_Classification/data/train/class_Q'+'/'+folder,img)
          print('stored at class_Q ')

        elif output_class=='class_R':
          cv2.imwrite('/content/drive/MyDrive/Number_Plate_Classification/data/train/class_R'+'/'+folder,img)
          print('stored at class_R ')

        elif output_class=='class_S':
          cv2.imwrite('/content/drive/MyDrive/Number_Plate_Classification/data/train/class_S'+'/'+folder,img)
          print('stored at class_S ')

        elif output_class=='class_T':
          cv2.imwrite('/content/drive/MyDrive/Number_Plate_Classification/data/train/class_T'+'/'+folder,img)
          print('stored at class_T ')

        elif output_class=='class_U':
          cv2.imwrite('/content/drive/MyDrive/Number_Plate_Classification/data/train/class_U'+'/'+folder,img)
          print('stored at class_U ')

        elif output_class=='class_V':
          cv2.imwrite('/content/drive/MyDrive/Number_Plate_Classification/data/train/class_V'+'/'+folder,img)
          print('stored at class_V ')

        elif output_class=='class_W':
          cv2.imwrite('/content/drive/MyDrive/Number_Plate_Classification/data/train/class_W'+'/'+folder,img)
          print('stored at class_W ')

        elif output_class=='class_X':
          cv2.imwrite('/content/drive/MyDrive/Number_Plate_Classification/data/train/class_X'+'/'+folder,img)
          print('stored at class_X ')

        elif output_class=='class_Y':
          cv2.imwrite('/content/drive/MyDrive/Number_Plate_Classification/data/train/class_Y'+'/'+folder,img)
          print('stored at class_Y ')

        elif output_class=='class_Z':
          cv2.imwrite('/content/drive/MyDrive/Number_Plate_Classification/data/train/class_Z'+'/'+folder,img)
          print('stored at class_Z ')

        



            # cv2.imwrite(os.path.join(desti_path+'/'+folder,i),transformed_img)
root_path='/content/drive/MyDrive/Number_Plate_Classification/data/naveen/naveen'
Saving_files(root_path)
